# Table of Contents
1. [Step 1: Prepare Data for Consumption](#step1)
1. [Step 2: Merge Datasets and Add Rows](#step2)
1. [Step 3: Extract Data from Features](#step3)
1. [Step 4: Add weather data](#step4)


<a id="step1"></a>
# Step 1: Prepare Data for Consumption

## 1.1 Import Libraries

In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_columns', None)
sns.set(style='white', context='notebook', palette='deep')

## 1.2 Load Data

In [68]:
df = pd.read_csv("./input/2018.csv")
df_extra = pd.read_csv("./input/df_with_neis2.csv")

c:\projects\python\autotel-ml\venv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 1.2 Prepare Data for Cosumption

In [69]:
df_extra.drop(['vehicle_id','shiryun_cat','avg_dist','billing_minutes','orig_addr','dest_addr',
               'start_dt','end_dt','orig_addr_c','dest_addr_c','orig_x_merc',
               'orig_y_merc','dest_x_merc','dest_y_merc','start_hour','start_dayofweek','start_month'],axis=1,inplace=True)

In [70]:
df = df.rename({'מספר רכב': 'Car id', 'מספר הזמנה': 'Order number','קטגוריית שריון':'Category',
                 'ממוצע מרחק לנסיעה':'Avg distance','דקות לחיוב':'Billing minutes','כתובת התחלה':'Start Address',
                 'כתובת סיום':'Dest Address','תאריך ושעה התחלת שריון':'Start time','תאריך ושעה סיום הזמנה':'End time'},
                axis=1) 

In [71]:
df_extra = df_extra.rename({'order_id': 'Order number'},axis=1) 

<a id="step2"></a>
# Step 2: Merge Datasets and Add Rows

## 2.1 Merge Datasets

In [72]:
df = pd.merge(df, df_extra, on=['Order number'], how='left')

## 2.2 Create New Rows
creating new rows from destination data (Dest Address, End time, dest_coord, dest_x, dest_y, dest_nei)

In [73]:
df_temp = df.copy()

In [74]:
df.drop(["Dest Address","End time","dest_coord","dest_x","dest_y","dest_nei"],axis=1,inplace=True)
df_temp.drop(["Start Address","Start time","orig_coord","orig_x","orig_y","orig_nei"],axis=1,inplace=True)

In [75]:
df = df.rename({"Start Address":"Address","Start time": "Time","orig_coord":"Coords","orig_x":"Lat","orig_y":"Lon","orig_nei":"neighborhood",},axis=1)
df_temp = df_temp.rename({"Dest Address":"Address","End time": "Time","dest_coord":"Coords","dest_x":"Lat","dest_y":"Lon","dest_nei":"neighborhood"},axis=1)

In [76]:
df = pd.concat([df,df_temp])

In [77]:
df.sort_values(["Order number","Time"],axis=0,inplace=True)

### [Delete] check for null and missing values

In [78]:
# Fill empty and NaNs values with NaN
df = df.fillna(np.nan)

In [79]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300004 entries, 0 to 150001
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Car id           300004 non-null  object 
 1   Order number     300004 non-null  int64  
 2   Category         300004 non-null  object 
 3   Avg distance     300004 non-null  int64  
 4   Billing minutes  275694 non-null  object 
 5   Address          286966 non-null  object 
 6   Time             286358 non-null  object 
 7   Unnamed: 0       272662 non-null  float64
 8   kmh              268428 non-null  float64
 9   Coords           272662 non-null  object 
 10  Lat              272662 non-null  float64
 11  Lon              272662 non-null  float64
 12  neighborhood     272662 non-null  object 
dtypes: float64(4), int64(2), object(7)
memory usage: 24.0+ MB


Car id                 0
Order number           0
Category               0
Avg distance           0
Billing minutes    24310
Address            13038
Time               13646
Unnamed: 0         27342
kmh                31576
Coords             27342
Lat                27342
Lon                27342
neighborhood       27342
dtype: int64

<a id="step3"></a>
# Step 3: Extract Data from Features

### Splitting address to country, city, street

In [80]:
split_col = df["Address"].str.split(",", expand=True)
split_col[:3]

,0,1,2,3
0,גורדון 86-88,תל אביב יפו,ישראל,None
0,הרב רובינשטיין 61,תל אביב יפו,ישראל,None
1,יעקב אפטר 3,תל אביב יפו,ישראל,None


In [81]:
df["Street"] = split_col[0]
df["City"] = split_col[1]
df["Country"] = split_col[2]

#### extracting street without numbers

In [82]:
df["Street_c"] = df['Street'].str.replace(r'(\d+)(-(\d+))*', r'').str.replace(r'\\', r'')

#### extract clean address for google api use

In [83]:
df['Address_c'] = df['Address'].str.replace(r'(\d+)-(\d+)', r'\1').str.replace(r'\\', r'')
df['Address_c2'] = df['Address'].str.replace(r'(\d+)(-(\d+))*', r'').str.replace(r'\\', r'')

### Extract date and time

In [84]:
df["Time"] = pd.to_datetime(df["Time"])

In [85]:
df["Date"] = df["Time"].dt.date
# df["time"] = df["Time"].dt.time
df["Year"] = df["Time"].dt.year
df["Month"] = df["Time"].dt.month
df["Day"] = df["Time"].dt.day
df["Hour"] = df["Time"].dt.hour
df["Minute"] = df["Time"].dt.minute

<a id="step4"></a>
# Step 4: Add weather data

In [87]:
weather = pd.read_csv("./input/ims_data_2017_2018.csv",na_values={
    "טמפרטורה(C°)": ["-"],
    "טמפרטורת מקסימום(C°)": ["-"],
    "טמפרטורת מינימום(C°)": ["-"],
    "לחות יחסית(%)": ["-"],
    "מהירות הרוח(m/s)": ["-"],
    "כיוון הרוח(מעלות)": ["-"],
    "מהירות רוח דקתית מקסימלית(m/s)": ["-"],
    "מהירות רוח 10 דקתית מקסימלית(m/s)": ["-"]
},dtype={
    "טמפרטורה(C°)": np.float64,
    "טמפרטורת מקסימום(C°)":np.float64,
    "טמפרטורת מינימום(C°)":np.float64,
    "לחות יחסית(%)": np.float64,
    "מהירות הרוח(m/s)":np.float64,
    'כמות גשם(מ"מ)': np.float64,
    "מהירות הרוח(m/s)": np.float64,
    "כיוון הרוח(מעלות)": np.float64,
    "מהירות רוח דקתית מקסימלית(m/s)": np.float64,
    "מהירות רוח 10 דקתית מקסימלית(m/s)": np.float64
})

In [88]:
weather = weather.rename({"שם תחנה":"station","תאריך":"Date","שעה- LST":"Time - LST","טמפרטורה(C°)":"Temprature",
                          "טמפרטורת מקסימום(C°)":"Max Temprature","טמפרטורת מינימום(C°)":"Min Temprature","לחות יחסית(%)":"Relative Humidity",
                          'כמות גשם(מ"מ)':"Amount of Rain","מהירות הרוח(m/s)":"Wind Speed","כיוון הרוח(מעלות)":"Wind Direction",
                          "מהירות רוח דקתית מקסימלית(m/s)":"Max Wind Speed per Min","מהירות רוח 10 דקתית מקסימלית(m/s)":"Max Win Speed per 10 Min"}
                         ,axis=1)

In [89]:
weather["Time - LST"] = pd.to_datetime(weather["Time - LST"])
weather["Date"] = pd.to_datetime(weather["Date"])

In [90]:
weather["Hour"] = weather["Time - LST"].dt.hour.astype(float)
weather["Date"] = weather["Date"].dt.date

In [100]:
# weather["Date"] = pd.to_datetime(weather["Date"])
# df["Date"] = pd.to_datetime(df["Date"])

In [141]:
# fill na values with previous values
weather.fillna(method='ffill',inplace=True)

In [95]:
weather.isna().sum()

station                     0
Date                        0
Time - LST                  0
Temprature                  0
Max Temprature              0
Min Temprature              0
Relative Humidity           0
Amount of Rain              0
Wind Speed                  0
Wind Direction              0
Max Wind Speed per Min      0
Max Win Speed per 10 Min    0
Hour                        0
dtype: int64

In [96]:
weather_hour_grouped = weather.groupby(["Date","Hour"]).agg({
        'Temprature':'mean', 'Max Temprature':'max','Min Temprature':'min',
        'Relative Humidity':'mean','Amount of Rain':'mean','Wind Speed':'mean',
        'Wind Direction':'mean','Max Wind Speed per Min':'max','Max Win Speed per 10 Min':'max'}).reset_index()

In [101]:
df = pd.merge(df,weather_hour_grouped,on=["Date","Hour"],how='left')

In [29]:
# d = df.groupby(["Date","Hour"]).size().reset_index()

In [30]:
# w = weather.groupby(["Date","Hour"]).size().reset_index()

In [31]:
#  tmp = pd.merge(d,w,on=["Date","Hour"],how="left")

In [32]:
df.drop(["Unnamed: 0"],axis=1, inplace=True)

In [45]:
# delete city != תל אביב יפו
df = df[df["City"] == ' תל אביב יפו']

In [48]:
df = df.round(2)
df.to_csv("./input/autotel.csv", encoding='utf-8-sig')

## Visualization

In [144]:
# counts = df['neighborhood'].value_counts()
# counts_more_then = df[df['neighborhood'].isin(counts[counts > 1000].index)]

In [145]:
# tmp = df.groupby(['Year', 'Month']).size().reset_index(name='Freq')

In [ ]:
# df.groupby(['Address', 'neighborhood']).size().reset_index(name='Freq')

In [146]:
# df[(df["Address_c2"] == ', תל אביב יפו, ישראל') & (df["neighborhood"] == "מונטיפיורי")]

In [147]:
#  df['neighborhood'].value_counts()

In [148]:

# df["value"] = df.groupby("name").transform(lambda x: x.fillna(x.mean()))